# Run train_base.py (LoRA / QLoRA) from Drive
本 Notebook 包含以下步骤：
1. 挂载 Google Drive（读取你已保存的 train_base.py / Configs / Data）
2. 安装依赖（transformers, datasets, peft, bitsandbytes, accelerate, huggingface_hub）
3. 设置 HF_TOKEN（从 Colab Secrets / 交互输入）
4. 做一个小规模 debug 子集并运行 train_base.py 进行快速 smoke-test
5.（可选）运行完整训练

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# 修改为你自己的项目根目录（与 train_base.py 写入位置一致）
BASE_DIR = "/content/drive/MyDrive/AIAA3102/Final_Project"   # <- 如果各位的路径不同，请修改
SCRIPTS_DIR = f"{BASE_DIR}/Scripts"
CONFIGS_DIR = f"{BASE_DIR}/Configs"
DATA_DIR = f"{BASE_DIR}/Data"
MODELS_DIR = f"{BASE_DIR}/Models"
RESULTS_DIR = f"{BASE_DIR}/Results"

print("BASE_DIR =", BASE_DIR)
!ls -la "{BASE_DIR}"


Mounted at /content/drive
BASE_DIR = /content/drive/MyDrive/AIAA3102/Final_Project
total 118
-rw------- 1 root root 75085 Nov  9 11:45 AIAA3102-FinalProject_Awareness_Ignorance.ipynb
drwx------ 2 root root  4096 Nov 13 10:19 Configs
drwx------ 2 root root  4096 Nov 13 10:19 Data
-rw------- 1 root root  8196 Nov 13 08:01 .DS_Store
drwx------ 2 root root  4096 Nov 14 09:53 Models
drwx------ 2 root root  4096 Nov 13 10:19 Scripts
-rw------- 1 root root 20387 Nov 24 09:06 test_results.json


# Install dependencies and import libraries

In [2]:
!pip install transformers datasets peft bitsandbytes accelerate huggingface_hub
# 需要一乃乃时间

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.1 MB/s eta 0:00:00


# Check Cuda

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model
import bitsandbytes as bnb

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


# Ensure the configs/script/py files

In [4]:
import os
paths = {
    "script": f"{SCRIPTS_DIR}/train_base.py",
    "configs": CONFIGS_DIR,
    "train": f"{DATA_DIR}/con_train.jsonl",
    "valid": f"{DATA_DIR}/con_valid.jsonl",
    "unknown": f"{DATA_DIR}/con_unknown_test.jsonl",
}
for k, p in paths.items():
    print(k, "exists:", os.path.exists(p), p)

# 打印前几行检查
print("\n--- train.jsonl (first 3 lines) ---")
!head -n 3 "{paths['train']}"
print("\n--- configs (list) ---")
!ls -la "{CONFIGS_DIR}"


script exists: True /content/drive/MyDrive/AIAA3102/Final_Project/Scripts/train_base.py
configs exists: True /content/drive/MyDrive/AIAA3102/Final_Project/Configs
train exists: True /content/drive/MyDrive/AIAA3102/Final_Project/Data/con_train.jsonl
valid exists: True /content/drive/MyDrive/AIAA3102/Final_Project/Data/con_valid.jsonl
unknown exists: True /content/drive/MyDrive/AIAA3102/Final_Project/Data/con_unknown_test.jsonl

--- train.jsonl (first 3 lines) ---
{"prompt": "I have suffered many things at home and school. We never went to the doctor to diagnose depression or anything like that, but I always feel like a part of my heart is missing. I try to fill it in with objects or, in this case, a woman.", "response": "There are a lot of ways to look at this. It sounds as if there is a part of you that is looking for support. I wonder what it is that you feel when you're not in a relationship?Because of the way you asked the question, I'm getting the impression that you are referring 

# Debug Dataset Generation

In [5]:
# Create tiny debug subsets to run a quick smoke-test (avoid long runs)
import shutil
from pathlib import Path
p_data = Path(DATA_DIR)
debug_train = p_data / "con_train_debug.jsonl"
debug_valid = p_data / "con_valid_debug.jsonl"

def subset(src, dst, n=20):
    with open(src, 'r', encoding='utf-8') as rf, open(dst, 'w', encoding='utf-8') as wf:
        for i, line in enumerate(rf):
            if i >= n:
                break
            wf.write(line)

subset(paths["train"], debug_train, n=20)
subset(paths["valid"], debug_valid, n=10)
print("Debug subsets created:", debug_train, debug_valid)
!wc -l "{debug_train}" "{debug_valid}"


Debug subsets created: /content/drive/MyDrive/AIAA3102/Final_Project/Data/con_train_debug.jsonl /content/drive/MyDrive/AIAA3102/Final_Project/Data/con_valid_debug.jsonl
   20 /content/drive/MyDrive/AIAA3102/Final_Project/Data/con_train_debug.jsonl
   10 /content/drive/MyDrive/AIAA3102/Final_Project/Data/con_valid_debug.jsonl
   30 total


**Training**

In [6]:
# 安装额外依赖
!pip install ipywidgets matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.4 MB/s eta 0:00:00


In [7]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import yaml
import json
from pathlib import Path
import subprocess
import traceback
import signal
import os
import re

class TrainingConfigGUI:
    def __init__(self, configs_dir):
        self.configs_dir = Path(configs_dir)
        self.widgets = {}
        self.output = widgets.Output()

    def load_configs(self):
        """加载现有配置"""
        training_cfg = yaml.safe_load(open(self.configs_dir / "training_args.yaml"))
        model_cfg = yaml.safe_load(open(self.configs_dir / "model_config.yaml"))
        return training_cfg, model_cfg

    def create_widgets(self):
        """创建交互式控件"""
        training_cfg, model_cfg = self.load_configs()

        # 基础训练参数
        self.widgets['num_train_epochs'] = widgets.IntSlider(
            value=training_cfg.get('num_train_epochs', 3),
            min=1, max=20, step=1,
            description='epoch:',
            style={'description_width': 'initial'}
        )

        self.widgets['learning_rate'] = widgets.FloatLogSlider(
            value=training_cfg.get('learning_rate', 5e-5),
            base=10, min=-6, max=-3,
            description='learning rate:',
            style={'description_width': 'initial'}
        )

        self.widgets['per_device_train_batch_size'] = widgets.IntSlider(
            value=training_cfg.get('per_device_train_batch_size', 2),
            min=1, max=16, step=1,
            description='batch size:',
            style={'description_width': 'initial'}
        )

        self.widgets['gradient_accumulation_steps'] = widgets.IntSlider(
            value=training_cfg.get('gradient_accumulation_steps', 1),
            min=1, max=16, step=1,
            description='gradient accumulation steps:',
            style={'description_width': 'initial'}
        )

        # LoRA参数
        lora_cfg = training_cfg.get('lora', {})
        self.widgets['use_lora'] = widgets.Checkbox(
            value=lora_cfg.get('use_lora', True),
            description='Use LoRA',
            style={'description_width': 'initial'}
        )

        self.widgets['lora_rank'] = widgets.IntSlider(
            value=lora_cfg.get('r', 8),
            min=4, max=64, step=4,
            description='LoRA rank:',
            style={'description_width': 'initial'}
        )

        self.widgets['lora_alpha'] = widgets.IntSlider(
            value=lora_cfg.get('lora_alpha', 32),
            min=8, max=128, step=8,
            description='LoRA Alpha:',
            style={'description_width': 'initial'}
        )

        # QLoRA参数
        qlora_cfg = training_cfg.get('qlora', {})
        self.widgets['use_qlora'] = widgets.Checkbox(
            value=qlora_cfg.get('use_qlora', False),
            description='Use QLoRA (4-bit)',
            style={'description_width': 'initial'}
        )

        # 数据参数
        self.widgets['max_train_samples'] = widgets.IntText(
            value=1000,
            description='Max Train Samples:',
            style={'description_width': 'initial'}
        )

        self.widgets['max_eval_samples'] = widgets.IntText(
            value=100,
            description='Max Eval Samples:',
            style={'description_width': 'initial'}
        )

        # 执行按钮
        self.widgets['run_button'] = widgets.Button(
            description='Start Training',
            button_style='success',
            icon='rocket'
        )
        self.widgets['run_button'].on_click(self.run_training)

        # 保存配置按钮
        self.widgets['save_button'] = widgets.Button(
            description='Save Settings',
            button_style='info',
            icon='save'
        )
        self.widgets['save_button'].on_click(self.save_configs)

        self.widgets['stop_button'] = widgets.Button(
            description='Stop Training',
            button_style='danger',
            icon='stop',
            disabled=True  # 初始时禁用
        )
        self.widgets['stop_button'].on_click(self.stop_training)

        self.training_process = None
        self.is_training = False

    def display_gui(self):
        """显示GUI界面"""
        self.create_widgets()

        # 组织布局
        training_params = widgets.VBox([
            self.widgets['num_train_epochs'],
            self.widgets['learning_rate'],
            self.widgets['per_device_train_batch_size'],
            self.widgets['gradient_accumulation_steps'],
        ])

        lora_params = widgets.VBox([
            self.widgets['use_lora'],
            self.widgets['lora_rank'],
            self.widgets['lora_alpha'],
            self.widgets['use_qlora'],
        ])

        data_params = widgets.VBox([
            self.widgets['max_train_samples'],
            self.widgets['max_eval_samples'],
        ])

        buttons = widgets.HBox([
            self.widgets['run_button'],
            self.widgets['stop_button'],
            self.widgets['save_button']
        ])

        # 使用选项卡组织界面
        tab = widgets.Tab()
        tab.children = [training_params, lora_params, data_params]
        tab.set_title(0, 'Training Parameters')
        tab.set_title(1, 'LoRA Parameters')
        tab.set_title(2, 'Data Parameters')

        # 显示界面
        display(widgets.VBox([tab, buttons, self.output]))

    def save_configs(self, button):
        """保存配置到文件"""
        with self.output:
            clear_output()

            # 更新配置
            training_cfg, model_cfg = self.load_configs()

            # 更新训练参数
            training_cfg['num_train_epochs'] = self.widgets['num_train_epochs'].value
            training_cfg['learning_rate'] = self.widgets['learning_rate'].value
            training_cfg['per_device_train_batch_size'] = self.widgets['per_device_train_batch_size'].value
            training_cfg['gradient_accumulation_steps'] = self.widgets['gradient_accumulation_steps'].value

            # 更新LoRA参数
            training_cfg['lora']['use_lora'] = self.widgets['use_lora'].value
            training_cfg['lora']['r'] = self.widgets['lora_rank'].value
            training_cfg['lora']['lora_alpha'] = self.widgets['lora_alpha'].value
            training_cfg['qlora']['use_qlora'] = self.widgets['use_qlora'].value

            # 保存文件
            with open(self.configs_dir / "training_args.yaml", 'w') as f:
                yaml.dump(training_cfg, f, default_flow_style=False)

            print("✅ Settings Saved!")
    def format_training_line(self, line, current_epoch):
      """格式化训练输出，移除重复信息"""
      line = line.strip()

      # 过滤掉不必要的信息
      if not line:
          return None

      # 过滤掉缓存警告（只显示一次）
      if 'use_cache=True' in line and 'incompatible' in line:
          if not hasattr(self, 'cache_warning_shown'):
              self.cache_warning_shown = True
              return "⚠️ " + line + "\n"
          return None

      # 进度百分比跟踪
      if '%|' in line:
          percent_match = re.search(r'(\d+)%\|', line)
          if percent_match:
              current_percent = int(percent_match.group(1))

              # 初始化
              if not hasattr(self, 'last_percent'):
                  self.last_percent = -1

              # 过滤重复百分比
              if current_percent == self.last_percent:
                  return None

              # 更新并显示
              self.last_percent = current_percent
              return line + "\n"


      # 默认返回原始行
      return line + "\n"

    def stop_training(self, button):
        """停止训练"""
        if self.training_process and self.is_training:
            try:
                # 发送终止信号
                self.training_process.terminate()

                # 等待进程结束（最多5秒）
                try:
                    self.training_process.wait(timeout=5)
                except subprocess.TimeoutExpired:
                    # 如果进程不响应，强制终止
                    self.training_process.kill()
                    self.training_process.wait()

                self.is_training = False
                self.widgets['stop_button'].disabled = True
                self.widgets['run_button'].disabled = False
                print("🛑 Training stopped by user")

            except Exception as e:
                print(f"❌ Error stopping training: {e}")
                # 强制恢复按钮状态
                self.is_training = False
                self.widgets['stop_button'].disabled = True
                self.widgets['run_button'].disabled = False
    def run_training(self, button):
        """运行训练"""
        with self.output:
            clear_output()
            print("🚀 Start Training...")
            print("Press 'Stop Training' to interrupt\n")
            # 更新按钮状态
            self.widgets['run_button'].disabled = True
            self.widgets['stop_button'].disabled = False
            self.is_training = True

            # 构建参数列表
            args = [
                "python", "-u", f"{SCRIPTS_DIR}/train_base.py",
                "--config_dir", f"{CONFIGS_DIR}",
                "--train_file", f"{DATA_DIR}/con_train.jsonl",
                "--valid_file", f"{DATA_DIR}/con_valid.jsonl",
                "--overwrite_output_dir",
                "--num_train_epochs", str(self.widgets['num_train_epochs'].value),
                "--learning_rate", str(self.widgets['learning_rate'].value),
                "--per_device_train_batch_size", str(self.widgets['per_device_train_batch_size'].value),
                "--gradient_accumulation_steps", str(self.widgets['gradient_accumulation_steps'].value),
                "--max_train_samples", str(self.widgets['max_train_samples'].value),
                "--max_eval_samples", str(self.widgets['max_eval_samples'].value),
                "--use_lora", str(self.widgets['use_lora'].value),
                "--use_qlora", str(self.widgets['use_qlora'].value),
                "--lora_rank", str(self.widgets['lora_rank'].value),
                "--lora_alpha", str(self.widgets['lora_alpha'].value)
            ]

            print("Command:", " ".join(args))
            print("="*60)

            try:
                # 使用 preexec_fn 确保子进程在单独的进程组中
                def preexec_function():
                    # 创建新的进程组
                    os.setpgrp()

                self.training_process = subprocess.Popen(
                    args,
                    stdout=subprocess.PIPE,
                    stderr=subprocess.STDOUT,
                    text=True,
                    bufsize=1,
                    universal_newlines=True,
                    preexec_fn=preexec_function  # 确保子进程在独立进程组
                )

                # 实时处理输出
                current_epoch = 0
                for line in self.training_process.stdout:
                    if not self.is_training:
                        # 用户请求停止，立即终止进程
                        print("\n🛑 Stopping training process...")
                        try:
                            # 终止整个进程组
                            os.killpg(os.getpgid(self.training_process.pid), signal.SIGTERM)
                            self.training_process.wait(timeout=5)
                        except:
                            try:
                                self.training_process.kill()
                                self.training_process.wait()
                            except:
                                pass
                        break

                    line = line.strip()
                    if not line:
                        continue

                    # 动态更新显示
                    formatted = self.format_training_line(line, current_epoch)
                    if formatted:
                        print(formatted, end='', flush=True)

                        # 更新epoch信息
                        if 'Epoch' in line and '/' in line:
                            try:
                                current_epoch = int(line.split('Epoch')[-1].split('/')[0].strip())
                            except:
                                pass

                # 检查进程是否还在运行
                if self.training_process.poll() is None:
                    # 进程仍在运行，强制终止
                    try:
                        os.killpg(os.getpgid(self.training_process.pid), signal.SIGKILL)
                        self.training_process.wait()
                    except:
                        pass

                # 获取返回码
                returncode = self.training_process.poll()

                # 恢复按钮状态
                self.widgets['run_button'].disabled = False
                self.widgets['stop_button'].disabled = True
                self.is_training = False

                if returncode == 0:
                    print("="*60)
                    print("✅ Training completed successfully!")
                elif returncode == -15 or returncode == 15:  # 被终止
                    print("="*60)
                    print("🛑 Training stopped by user")
                else:
                    print("="*60)
                    print(f"❌ Training failed (code: {returncode})")

            except Exception as e:
                print(f"❌ Error: {e}")
                import traceback
                traceback.print_exc()
                # 恢复按钮状态
                self.widgets['run_button'].disabled = False
                self.widgets['stop_button'].disabled = True
                self.is_training = False

# 使用GUI
gui = TrainingConfigGUI(CONFIGS_DIR)
gui.display_gui()

In [ ]:
# 在 notebook cell 中运行（替换 output_dir 为你的训练输出）
%load_ext tensorboard
%tensorboard --logdir "/content/drive/MyDrive/AIAA3102/Final_Project/Models/tinyllama_ai_finetuned/tb_runs"

In [ ]:
!cp -r /content/models/tinyllama_ai_finetuned /content/drive/MyDrive/AIAA3102/Final_Project/Models

In [ ]:
!rsync -av --delete --progress /content/drive/MyDrive/AIAA3102/Final_Project/Models/tinyllama_ai_finetuned/ /content/models/tinyllama_ai_finetuned/

sending incremental file list

sent 1,130 bytes  received 15 bytes  2,290.00 bytes/sec
total size is 107,257,357  speedup is 93,674.55


# Assessment with Safety and Domain Check

In [11]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig
import os, json
import torch
import yaml
from tqdm import tqdm


class ModelTester:
    def __init__(self, configs_dir, data_dir, base_dir, results_dir):
        self.configs_dir = configs_dir
        self.data_dir = data_dir
        self.base_dir = base_dir
        self.results_dir = results_dir
        self.model = None
        self.tokenizer = None

    def _safety_and_domain_check(self, user_input: str):
        """
        Check if input contains dangerous content or is out-of-domain.
        Returns (should_block: bool, response_message: str)
        All messages are in English.
        """
        user_input_lower = user_input.lower().strip()

        # Dangerous keywords (English only, for your use case)
        DANGEROUS_KEYWORDS = {
            'suicide', 'kill myself', 'end my life', 'want to die', 'die alone',
            'murder', 'homicide', 'self harm', 'cut myself', 'overdose',
            'no reason to live', 'hurt myself', 'suicidal thoughts',
            'can\'t go on', 'better off dead'
        }

        # Out-of-domain keywords (clear non-counseling topics)
        OUT_OF_DOMAIN_KEYWORDS = {
            'code', 'python', 'java', 'javascript', 'c++', 'html', 'css', 'sql',
            'math', 'calculate', 'solve', 'equation', 'derivative', 'integral', 'algebra',
            'weather', 'forecast', 'stock', 'price', 'bitcoin', 'crypto',
            'translate', 'summarize', 'write a poem', 'build a website',
            'how to install', 'debug', 'error message', 'algorithm'
        }

        # Check dangerous content first (higher priority)
        for phrase in DANGEROUS_KEYWORDS:
            if phrase in user_input_lower:
                return True, (
                    "I'm really concerned about what you're going through. "
                    "Please reach out to a mental health professional or contact a crisis hotline immediately. "
                    "In China, you can call or text 119. "
                    "Your life matters, and help is available."
                )

        # Check out-of-domain content
        for phrase in OUT_OF_DOMAIN_KEYWORDS:
            if phrase in user_input_lower:
                return True, (
                    "I'm designed to provide emotional support and counseling-related guidance only. "
                    "This question is outside my area of expertise."
                )

        return False, ""

    def load_model(self):
        """Load model and tokenizer"""
        cfg = yaml.safe_load(open(f"{self.configs_dir}/training_args.yaml"))
        outdir = cfg.get("output_dir", f"{self.base_dir}/models/finetuned_model")

        print(f"🔍 Checking model directory: {outdir}")
        if not os.path.exists(outdir):
            raise FileNotFoundError(f"Model directory not found: {outdir}")

        try:
            peft_config = PeftConfig.from_pretrained(outdir)
            is_lora = True
            base_model_name = peft_config.base_model_name_or_path
            print(f"✅ Detected LoRA model, base model: {base_model_name}")
        except:
            is_lora = False
            base_model_name = cfg.get("model_name_or_path", "TinyLlama/TinyLlama_v1.1")
            print(f"✅ Loading full model: {base_model_name}")

        self.tokenizer = AutoTokenizer.from_pretrained(base_model_name)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        if is_lora:
            base_model = AutoModelForCausalLM.from_pretrained(
                base_model_name,
                device_map="auto",
                torch_dtype=torch.float16,
                trust_remote_code=True
            )
            self.model = PeftModel.from_pretrained(base_model, outdir)
            print("✅ LoRA adapter loaded successfully")
        else:
            self.model = AutoModelForCausalLM.from_pretrained(
                outdir,
                device_map="auto",
                torch_dtype=torch.float16,
                trust_remote_code=True
            )
            print("✅ Full model loaded successfully")

        print(f"📊 Model parameters: {self.model.num_parameters():,}")
        print(f"💻 Model device: {self.model.device}")

    def generate_response(self, prompt, generation_config=None):
        """Generate model response"""
        if generation_config is None:
            generation_config = {
                "max_new_tokens": 256,
                "do_sample": True,
                "temperature": 1.0,
                "top_p": 1.0,
                "repetition_penalty": 1.1,
                "pad_token_id": self.tokenizer.eos_token_id
            }

        inputs = self.tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(self.model.device)

        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                **generation_config
            )

        full_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated_text = full_text[len(prompt):].strip()
        return full_text, generated_text

    def test_validation_set(self, num_samples=10, save_results=False):
        """Test on validation set"""
        print(f"\n🧪 Starting validation test ({num_samples} samples)")

        with open(f"{self.data_dir}/con_valid.jsonl", 'r', encoding='utf-8') as f:
            lines = [json.loads(l) for l in f][:num_samples]

        results = []

        for i, ex in enumerate(tqdm(lines, desc="Testing")):
            prompt_text = ex['prompt']
            expected_response = ex.get('response', '')

            # Safety/domain check
            blocked, safety_msg = self._safety_and_domain_check(prompt_text)
            if blocked:
                full_text = f"Question:\n{prompt_text}\nAnswer:\n{safety_msg}"
                generated_text = safety_msg
            else:
                prompt = f"Question:\n{prompt_text}\nAnswer:\n"
                try:
                    full_text, generated_text = self.generate_response(prompt)
                except Exception as e:
                    generated_text = f"[Generation failed: {str(e)}]"
                    full_text = prompt + generated_text

            result = {
                "id": i,
                "prompt": prompt_text,
                "expected": expected_response,
                "generated": generated_text,
                "full_output": full_text
            }
            results.append(result)

            print(f"\n{'='*60}")
            print(f"📝 Sample {i+1}:")
            print(f"❓ Question: {prompt_text[:100]}...")
            print(f"🤖 Generated: {generated_text}")
            print(f"✅ Expected: {expected_response}")

        if save_results:
            output_file = f"{self.results_dir}/test_results.json"
            os.makedirs(self.results_dir, exist_ok=True)
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(results, f, ensure_ascii=False, indent=2)
            print(f"\n💾 Results saved to: {output_file}")

        return results

    def interactive_mode(self):
        """Interactive testing mode"""
        print("\n🎮 Entering interactive mode (type 'quit' to exit)")

        generation_configs = {
            "creative": {
                "max_new_tokens": 300,
                "do_sample": True,
                "temperature": 0.8,
                "top_p": 0.9,
                "top_k": 50
            },
            "precise": {
                "max_new_tokens": 200,
                "do_sample": False,
                "temperature": 1.0,
                "top_p": 1.0
            },
            "balanced": {
                "max_new_tokens": 256,
                "do_sample": True,
                "temperature": 0.7,
                "top_p": 0.9
            }
        }

        current_mode = "balanced"

        while True:
            try:
                user_input = input(f"\n💬 Enter your question [{current_mode} mode]: ").strip()

                if user_input.lower() in ['quit', 'exit']:
                    break
                elif user_input.lower() == 'mode':
                    print("Available modes: creative, precise, balanced")
                    new_mode = input("Select mode: ").strip()
                    if new_mode in generation_configs:
                        current_mode = new_mode
                        print(f"✅ Switched to {current_mode} mode")
                    continue
                elif not user_input:
                    continue

                # 🔒 Safety and domain check
                blocked, safety_msg = self._safety_and_domain_check(user_input)
                if blocked:
                    print(f"📝 {safety_msg}")
                    continue

                prompt = f"Question:\n{user_input}\nAnswer:\n"
                full_text, generated_text = self.generate_response(
                    prompt,
                    generation_configs[current_mode]
                )

                print(f"\n🤖 Model Response ({current_mode} mode):")
                print(f"📝 {generated_text}")

            except KeyboardInterrupt:
                print("\n👋 Exiting interactive mode")
                break
            except Exception as e:
                print(f"❌ Generation failed: {e}")


# ==========================
# Example main() – adjust paths as needed
# ==========================
def main():
    tester = ModelTester(CONFIGS_DIR, DATA_DIR, BASE_DIR, RESULTS_DIR)

    try:
        tester.load_model()
        tester.test_validation_set(num_samples=5, save_results=True)
        #tester.interactive_mode()
    except Exception as e:
        print(f"❌ Error: {e}")


if __name__ == "__main__":
    main()

🔍 Checking model directory: models/tinyllama_ai_finetuned
✅ Detected LoRA model, base model: TinyLlama/TinyLlama_v1.1
✅ LoRA adapter loaded successfully
📊 Model parameters: 1,102,301,184
💻 Model device: cuda:0

🧪 Starting validation test (5 samples)


Testing:  20%|██        | 1/5 [00:12<00:51, 12.90s/it]


📝 Sample 1:
❓ Question: I'm always listening to my husband, but it feels like he never listens to me. 

How do I get my hu...
🤖 Generated: For starters you've hit upon what's been bothering couples for at least a thousand years...
He doesn't want to listen to what women have to say! That said, being an individual is only possible if you care about another individual person, and there are a lot of people out here who care (mostly males).
This means there has got to be some serious, ongoing reason why your significant other won't pay attention when you talk directly with them. It could be that his listening muscles aren't as strong as yours so he will let you slide by when talking directly to him. Maybe there is such a deep-seated fear of what he might think or say that it makes him not pay full attention. Or perhaps he's not sure how much information to allow into his mind, and so the tension could grow. For now though, concentrate on you're talking to yourself. You're saying and think

Testing:  40%|████      | 2/5 [00:25<00:38, 12.87s/it]


📝 Sample 2:
❓ Question: I've been with a man for four years. For the last year, he has said he is done, but he still talks, ...
🤖 Generated: Hi, Shelley! 14 months together...that's an amazing relationship considering everyone says relationships are hard but you guys have managed to make it so far.
Your partner seems like an incredible guy but when asked why doesn't he move on you haven't heard any answers that help you feel better about your partner who still refuses to do what he could be doing. It gets harder everyday because of this and I get even more frustrated. It is possible that sometimes guys will just shut down and not give in and not want to say the word "quit". But if that doesn't really do anything please see if there are other questions we can provide to aid in finding out how to approach him. Maybe the best thing he would tell someone, is "If you will ever say "stay", there will be no turning back." This helps the person to put away all thoughts of leaving and stay rat

...


📝 Sample 3:
❓ Question: After first meeting the client, what is the process that a counselor facilitates?...
🤖 Generated: After meeting the client for the first time during the intake interview, or after a face to face or phone or group therapy appointment. My counseling style will always be collaborative and goal oriented so you come to the counselor sessions with a preliminary sense of what you would like to do. The individualized work that begins each week usually has some type of goal or outcome in mind.
If the counselor is an active member of your therapy it will usually take place within a group. This type of work is often less common then in other types of therapies or sessions. In our offices we will offer a variety of opportunities in one of our offices to meet once per week for at least half an hour at 11 AM.
The counselor is always available to make suggestions on how to implement any part or section of the process if the client is already making some progress on his/her ow

Testing:  80%|████████  | 4/5 [00:51<00:12, 12.91s/it]


📝 Sample 4:
❓ Question: I feel like every time I do something someone asks me to, I never fully meet what they want. I feel ...
🤖 Generated: Well, maybe you are doing more than meets the eye there, you may be understimulated? Not having met exactly the needs of others for some reason I'm guessing you're not quite satisfied. I wonder if a great challenge has been introduced by this interaction on theirs part(s). As the saying goes... we may come across all sorts of obstacles!
And on another note! Not wanting and meeting your own needs is indeed a bit of an unusual approach. How about if you asked everyone if they needed you to go help them with something. Maybe ask for feedback or a bit of self-examination. Is this how you would feel in this scenario if your partner did it... or anyone else? What do you feel? Then be the hero - the one who steps forward... and does the job!
I am a writer/performer teaching writing courses and workshops on the East coast. I have studied theatre writing 

Testing: 100%|██████████| 5/5 [01:04<00:00, 12.96s/it]


📝 Sample 5:
❓ Question: It's been almost a year since my ex-boyfriend broke up with me after he cheated on me many times. I ...
🤖 Generated: In your example, you have two choices you can make. You can either look at things as healing or haunting yourself in a negative way and in the opposite direction. Which one do you think is the best for yourself? The fact he helped you through your feelings by giving financial support makes you feel gratitude towards him. On the other hand, he betrayed his promise to love and protect you. There are no good or bad examples; it just depends on the meaning.
However, in order of gratitude to forgiveness, I highly recommend that you look into Forgiveness Therapy. From my own personal experience, it's something I needed to do in my life and in some cases I never got around to it. Most people don't. After my workshops, I have a greater understanding of the situation that led to my betrayal. My feelings started healing. As your experience evolves more and

In [12]:
tester = ModelTester(CONFIGS_DIR, DATA_DIR, BASE_DIR, RESULTS_DIR)
tester.load_model()
tester.interactive_mode()

🔍 Checking model directory: models/tinyllama_ai_finetuned
✅ Detected LoRA model, base model: TinyLlama/TinyLlama_v1.1
✅ LoRA adapter loaded successfully
📊 Model parameters: 1,102,301,184
💻 Model device: cuda:0

🎮 Entering interactive mode (type 'quit' to exit)

💬 Enter your question [balanced mode]: I want to suicide
📝 I'm really concerned about what you're going through. Please reach out to a mental health professional or contact a crisis hotline immediately. In China, you can call or text 119. Your life matters, and help is available.

💬 Enter your question [balanced mode]: I want you to teach me math
📝 I'm designed to provide emotional support and counseling-related guidance only. This question is outside my area of expertise.

💬 Enter your question [balanced mode]: quit


# Quick Assessment

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama_v1.1")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 示例 prompt/response
prompt = "### 问：\nI'm so sad because my best friend left.？\n### 答：\n"
response = "I'm sorry to hear..."

# tokenization as our new function does
p_ids = tokenizer(prompt, add_special_tokens=False)["input_ids"]
r_ids = tokenizer(response, add_special_tokens=False)["input_ids"]
max_length = 128
full = p_ids + r_ids
if len(full) > max_length:
    full = full[-max_length:]
    if len(r_ids) >= max_length:
        resp_start = 0
    else:
        resp_start = max(0, len(full) - len(r_ids))
else:
    resp_start = len(p_ids)

labels = [-100] * len(full)
for i in range(resp_start, len(full)):
    labels[i] = full[i]

pad_len = max_length - len(full)
input_ids = full + [tokenizer.pad_token_id] * pad_len
labels = labels + [-100] * pad_len

print("decoded:", tokenizer.decode(input_ids[:len(full)]))
print("input_ids (head):", input_ids[:len(full)])
print("labels (head):", labels[:len(full)])
print("labels contains -100?", -100 in labels)


decoded: ### 问：
I'm so sad because my best friend left.？
### 答：
 I'm sorry to hear...
input_ids (head): [835, 29871, 31658, 30383, 13, 29902, 29915, 29885, 577, 14610, 1363, 590, 1900, 5121, 2175, 29889, 30882, 13, 2277, 29937, 29871, 234, 176, 151, 30383, 13, 306, 29915, 29885, 7423, 304, 8293, 856]
labels (head): [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 306, 29915, 29885, 7423, 304, 8293, 856]
labels contains -100? True


### Baseline Tinyllama without LoRA


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Define the base model name
base_model_name = "TinyLlama/TinyLlama_v1.1"

print(f"Loading tokenizer for base model: {base_model_name}")
base_tokenizer = AutoTokenizer.from_pretrained(base_model_name)
print("Base tokenizer loaded.")

print(f"Loading base model: {base_model_name}")
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    dtype=torch.float16,
    trust_remote_code=True
)
print("Base model loaded.")
print(f"Base model device: {base_model.device}")
print(f"Base model parameters: {base_model.num_parameters():,}")

Loading tokenizer for base model: TinyLlama/TinyLlama_v1.1
Base tokenizer loaded.
Loading base model: TinyLlama/TinyLlama_v1.1
Base model loaded.
Base model device: cuda:0
Base model parameters: 1,100,048,384


In [ ]:
import json
import torch
from tqdm import tqdm

def generate_response_base(model, tokenizer, prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_p=0.9, repetition_penalty=1.1):
    """Generates a response using the base model."""
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    generation_config = {
        "max_new_tokens": max_new_tokens,
        "do_sample": do_sample,
        "temperature": temperature,
        "top_p": top_p,
        "repetition_penalty": repetition_penalty,
        "pad_token_id": tokenizer.eos_token_id
    }

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            **generation_config
        )

    full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_text = full_text[len(prompt):]
    return generated_text

# 1. Load the con_valid.jsonl dataset and select the first 5 samples
validation_data_path = f"{DATA_DIR}/con_valid.jsonl"
base_model_results = []

with open(validation_data_path, 'r', encoding='utf-8') as f:
    validation_samples = [json.loads(line) for line in f][:5]

# 2. Iterate through the selected validation samples, format the prompt, and generate responses
print("\n🚀 Generating responses with the original TinyLlama model...")
for i, sample in enumerate(tqdm(validation_samples, desc="Base model inference")):
    prompt_text = f"问：\n{sample['prompt']}\n答：\n"
    generated_response = generate_response_base(base_model, base_tokenizer, prompt_text)

    base_model_results.append({
        "id": i,
        "prompt": sample['prompt'],
        "expected": sample.get('response', ''),
        "generated": generated_response.strip()
    })

    print(f"\n{'='*60}")
    print(f"📝 Sample {i+1} (Base Model):")
    print(f"❓ Question: {sample['prompt'][:100]}...")
    print(f"🤖 Generated Response: {generated_response.strip()}")
    print(f"✅ Expected Response: {sample.get('response', '')}")

print("\n✅ Base model inference complete.")



🚀 Generating responses with the original TinyLlama model...


Base model inference:  20%|██        | 1/5 [00:17<01:11, 17.96s/it]


📝 Sample 1 (Base Model):
❓ Question: I'm always listening to my husband, but it feels like he never listens to me. 

How do I get my hu...
🤖 Generated Response: Husbands and wives are not robots. They have feelings, too. If you want your husband to listen to you, you need to give him a reason to. If he thinks that you just don't care about him enough, then that is his problem. If you don't want to argue with him, but want to be heard, then you have to make sure you say what you mean.
If you want your husband to listen to you, then he needs to understand your point of view, or at least try.
There are three things you can do:
1. Start by saying "I love you" to your husband every morning (even if it's not true).
2. Ask for what you want in your marriage.
3. If you know something is bothering him, ask him if he wants to talk about it.
You can start by saying "I love you."
You can ask for what you want in your marriage.
You can ask your husband if he wants to talk about it.
You can ask him

Base model inference:  40%|████      | 2/5 [00:34<00:51, 17.09s/it]


📝 Sample 2 (Base Model):
❓ Question: I've been with a man for four years. For the last year, he has said he is done, but he still talks, ...
🤖 Generated Response: His words do not match his actions. I love this man, but it's hurting so much.
She has been married for two years. He has been with her for six months. She wants to be single. He wants to get married again. They both have children.
She has been married for two years. He has been with her for six months. She wants to be single. He wants to get married again. They both have children.
He said he was going to get married in September, but never did. The wedding never happened. He said he would wait until next year but never did. He also said that he would never get married again because of how much time they had spent apart since their first meeting.
He said he was going to get married in September, but never did. The wedding never happened. He said he would wait until next year but never did. He also said that he would never ge

...


📝 Sample 3 (Base Model):
❓ Question: After first meeting the client, what is the process that a counselor facilitates?...
🤖 Generated Response: The counseling process begins with a thorough assessment of the client. The counselor uses this information to develop an individualized treatment plan. The plan may include counseling, referrals for appropriate services and follow-up sessions as necessary.
When a client comes to see you, how do you assess their level of distress? What are some signs that a person has been affected by a traumatic event or may be at risk of developing PTSD?
答 e
Assessment includes:
What are the client's goals?
What are the client's symptoms?
Is there a history of substance abuse? If so, how many days/weeks in the last month have you used alcohol or drugs?
Does the client appear anxious, depressed or agitated? Are they irritable? Do they seem to be unable to concentrate?
Are there any recent events or circumstances that could be related to their current problems

Base model inference:  80%|████████  | 4/5 [00:51<00:11, 11.37s/it]


📝 Sample 4 (Base Model):
❓ Question: I feel like every time I do something someone asks me to, I never fully meet what they want. I feel ...
🤖 Generated Response: I feel like I have to do everything for everyone.
I feel like people always ask me to help them but I don't really know how to help anyone because I'm not sure if they are worth helping. So I feel like I'm just giving up on myself.
I feel like sometimes, I'm just so busy doing my own thing that I forget about the things other people need. Like, when I'm having a bad day, I'll just go back to what I was doing before and that will make me happy. But then I feel guilty when I'm having a good day.
答
I feel like sometimes I have no control over myself. I can be really hard on myself sometimes. I feel like there is a lot of negativity in my head that I have to try to get rid of. And I think that's why I feel like I can't let people into my life too easily. I feel like I can only take so much. I also feel like I don't know who I am

Base model inference: 100%|██████████| 5/5 [00:59<00:00, 11.94s/it]


📝 Sample 5 (Base Model):
❓ Question: It's been almost a year since my ex-boyfriend broke up with me after he cheated on me many times. I ...
🤖 Generated Response: I'm so sorry that you've gone through such a traumatic experience in your life. It's understandable why you feel so upset. But also I think you'll be glad to know that you are not alone. I can tell you that it feels so much better to let go of the guilt, and forgive yourself, and move on from the betrayal. This is especially important as you continue to heal.
You've mentioned how you feel guilty because you're still mad at him. This is a common reaction to betrayal, and is a normal feeling. You should probably remind yourself that your feelings don't mean anything to him. Your ex is not going to change his ways or think differently about you. He probably just doesn't care what happens to you.
The first step in moving forward is forgiving yourself for the hurt you've felt. It may take some time, but it will help you to let go

### LoRA Performance

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig
import os, json
import torch
import yaml
from tqdm import tqdm

class ModelTester:
    def __init__(self, configs_dir, data_dir, base_dir, results_dir):
        self.configs_dir = configs_dir
        self.data_dir = data_dir
        self.base_dir = base_dir
        self.results_dir = results_dir
        self.model = None
        self.tokenizer = None

    def load_model(self):
        """加载模型和tokenizer"""
        # 读取配置
        cfg = yaml.safe_load(open(f"{self.configs_dir}/training_args.yaml"))
        outdir = cfg.get("output_dir", f"{self.base_dir}/models/finetuned_model")

        print(f"🔍 检查模型目录: {outdir}")
        if not os.path.exists(outdir):
            raise FileNotFoundError(f"模型目录不存在: {outdir}")

        # 检查是否是LoRA模型
        try:
            peft_config = PeftConfig.from_pretrained(outdir)
            is_lora = True
            base_model_name = peft_config.base_model_name_or_path
            print(f"✅ 检测到LoRA模型，基础模型: {base_model_name}")
        except:
            is_lora = False
            base_model_name = cfg.get("model_name_or_path", "TinyLlama/TinyLlama_v1.1")
            print(f"✅ 加载完整模型: {base_model_name}")

        # 加载tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(base_model_name)

        # 加载模型
        if is_lora:
            # LoRA模型加载方式
            base_model = AutoModelForCausalLM.from_pretrained(
                base_model_name,
                device_map="auto",
                torch_dtype=torch.float16,
                trust_remote_code=True
            )
            self.model = PeftModel.from_pretrained(base_model, outdir)
            print("✅ LoRA适配器加载完成")
        else:
            # 完整模型加载方式
            self.model = AutoModelForCausalLM.from_pretrained(
                outdir,
                device_map="auto",
                torch_dtype=torch.float16,
                trust_remote_code=True
            )
            print("✅ 完整模型加载完成")

        print(f"📊 模型参数数量: {self.model.num_parameters():,}")
        print(f"💻 模型设备: {self.model.device}")

    def generate_response(self, prompt, generation_config=None):
        """生成回复"""
        if generation_config is None:
            generation_config = {
                "max_new_tokens": 256,
                "do_sample": True,
                "temperature": 0.1,
                "top_p": 0.5,
                "repetition_penalty": 1.1,
                "pad_token_id": self.tokenizer.eos_token_id
            }

        # 编码输入
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)

        # 生成
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                **generation_config
            )

        # 解码输出
        full_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated_text = full_text[len(prompt):]

        return full_text, generated_text

    def test_validation_set(self, num_samples=10, save_results=False):
        """测试验证集"""
        print(f"\n🧪 开始验证集测试 ({num_samples}个样本)")

        # 加载验证数据
        with open(f"{self.data_dir}/con_valid.jsonl", 'r', encoding='utf-8') as f:
            lines = [json.loads(l) for l in f][:num_samples]

        results = []

        for i, ex in enumerate(tqdm(lines, desc="测试进度")):
            prompt = f"问：\n{ex['prompt']}\n答：\n"
            expected_response = ex.get('response', '')

            try:
                full_text, generated_text = self.generate_response(prompt)

                result = {
                    "id": i,
                    "prompt": ex['prompt'],
                    "expected": expected_response,
                    "generated": generated_text,
                    "full_output": full_text
                }
                results.append(result)

                # 打印结果
                print(f"\n{'='*60}")
                print(f"📝 样本 {i+1}:")
                print(f"❓ 问题: {ex['prompt'][:100]}...")
                print(f"🤖 生成回答: {generated_text.strip()}")
                print(f"✅ 期望回答: {expected_response}")

            except Exception as e:
                print(f"❌ 样本 {i+1} 生成失败: {e}")
                continue

        # 保存结果
        if save_results:
            output_file = f"{self.results_dir}/test_results.json"
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(results, f, ensure_ascii=False, indent=2)
            print(f"\n💾 测试结果已保存至: {output_file}")

        return results

    def interactive_mode(self):
        """交互式测试模式"""
        print("\n🎮 进入交互模式 (输入 'quit' 退出)")

        generation_configs = {
            "creative": {
                "max_new_tokens": 300,
                "do_sample": True,
                "temperature": 0.8,
                "top_p": 0.9,
                "top_k": 50
            },
            "precise": {
                "max_new_tokens": 200,
                "do_sample": False,  # 贪婪解码
                "temperature": 1.0,
                "top_p": 1.0
            },
            "balanced": {
                "max_new_tokens": 256,
                "do_sample": True,
                "temperature": 0.7,
                "top_p": 0.9
            }
        }

        current_mode = "balanced"

        while True:
            try:
                user_input = input(f"\n💬 请输入问题 [{current_mode}模式]: ").strip()

                if user_input.lower() in ['quit', 'exit', '退出']:
                    break
                elif user_input.lower() == 'mode':
                    # 切换生成模式
                    print("可用模式: creative, precise, balanced")
                    new_mode = input("选择模式: ").strip()
                    if new_mode in generation_configs:
                        current_mode = new_mode
                        print(f"✅ 切换到 {current_mode} 模式")
                    continue
                elif not user_input:
                    continue

                prompt = f"问：\n{user_input}\n答：\n"
                full_text, generated_text = self.generate_response(
                    prompt,
                    generation_configs[current_mode]
                )

                print(f"\n🤖 模型回答 ({current_mode}模式):")
                print(f"📝 {generated_text.strip()}")

            except KeyboardInterrupt:
                print("\n👋 退出交互模式")
                break
            except Exception as e:
                print(f"❌ 生成失败: {e}")

def main():
    """主函数"""
    tester = ModelTester(CONFIGS_DIR, DATA_DIR, BASE_DIR, RESULTS_DIR)

    try:
        # 1. 加载模型
        tester.load_model()

        # 2. 验证集测试
        tester.test_validation_set(num_samples=5, save_results=True)

        # 3. 交互模式
        #tester.interactive_mode()

    except Exception as e:
        print(f"❌ 错误: {e}")

if __name__ == "__main__":
    main()

🔍 检查模型目录: models/tinyllama_ai_finetuned
✅ 检测到LoRA模型，基础模型: TinyLlama/TinyLlama_v1.1
✅ LoRA适配器加载完成
📊 模型参数数量: 1,102,301,184
💻 模型设备: cuda:0

🧪 开始验证集测试 (5个样本)


测试进度:  20%|██        | 1/5 [00:17<01:09, 17.32s/it]


📝 样本 1:
❓ 问题: I'm always listening to my husband, but it feels like he never listens to me. 

How do I get my hu...
🤖 生成回答: It is a good idea to ask your husband what he wants you to do and how you can help him achieve his goals. If you are not sure about what he wants or needs from you, then you should ask him directly. You may also want to consider asking your husband if there is anything that he would like you to do for him. This will give you an idea of whether or not you two are on the same page when it comes to communication. Finally, try to be patient with your husband as he may take some time to understand what you are saying. Remember that he is trying to understand you as well. Keep in mind that communication is a two-way street and both parties need to be willing to listen to each other. If you can work together to find common ground, then you will have a better chance of resolving any issues that arise. Best of luck!
What is the best way to communicate with someone who is 

测试进度:  40%|████      | 2/5 [00:32<00:47, 15.78s/it]


📝 样本 2:
❓ 问题: I've been with a man for four years. For the last year, he has said he is done, but he still talks, ...
🤖 生成回答: It sounds like you are in a very difficult situation. It may be that your partner is simply not ready to move on from you. If he is not ready to move on, then you should respect his wishes. However, if he is ready to move on, then you should respect his wishes as well. It would be best to talk to him about what you want and see if he is willing to work together to find a solution. Good luck!
What is the difference between a relationship and a marriage?
How can I get over my ex-boyfriend?
Is it normal to feel sad when someone you care about dies?
How do I stop feeling guilty for being single?
How do I know if I am in love or just infatuated?
How do I stop feeling like I have to prove myself?
How do I stop feeling like I need to prove myself? I'm a college student who feels like I need to prove myself all the time. I'm always trying to do better than everyone els

...

测试进度:  80%|████████  | 4/5 [01:01<00:15, 15.06s/it]


📝 样本 4:
❓ 问题: I feel like every time I do something someone asks me to, I never fully meet what they want. I feel ...
🤖 生成回答: It's not uncommon for people to ask us to do things and then not follow through on the request. It can be frustrating because we want to help others but we don't know how to do so. The best way to handle this is to be honest about your feelings. If you are feeling frustrated or disappointed in yourself, then you will probably be more open to accepting help from others. You may also find that other people are willing to help you out if you let them know what you need. Finally, try to remember that there are many ways to accomplish a task and sometimes it's just easier to delegate tasks to others. Good luck!
What does it mean when someone says "I love you" but doesn't mean it?
It means that they don't really care about you. They only say those words to make themselves feel better.
How do I stop myself from saying yes to everything?
Say no to everything. Saying ye

测试进度: 100%|██████████| 5/5 [01:14<00:00, 14.92s/it]


📝 样本 5:
❓ 问题: It's been almost a year since my ex-boyfriend broke up with me after he cheated on me many times. I ...
🤖 生成回答: I'm sorry you are going through such a difficult time. It sounds like you are trying to move forward in your life while also dealing with the pain of your ex-boyfriend's betrayal. I would recommend that you seek counseling from someone who specializes in working with trauma. You may be able to work through some of these issues with the help of a therapist. If you decide to go ahead with therapy, I would suggest that you bring up the topic of your ex-boyfriend's betrayal during your first session with your therapist. It could be helpful to share your thoughts and feelings with someone who will listen without judgement. I hope that you find peace and closure in your journey. Best of luck!
Kimberly L. Hicks, LCSW, BC-DMT, CADC, MBA, NCC, AAMFT
Kimberly L. Hicks, LCSW, BC-DMT, CADC, MBA, NCC, AAMFT, is a Licensed Clinical Social Worker, Board Certified Addictionolo


💾 测试结果已保存至: /content/drive/MyDrive/AIAA3102/Final_Project/Results/test_results.json


### ROUGE-L scores & LLM-as-Judge

In [ ]:
import json
import pandas as pd
from tqdm import tqdm
from openai import OpenAI
import re
# 配置客户端
key = "sk-GslHHRxrgakQY0XKiCNP2ZoFDU53bKtEuexTLjG3mmGflMlW"  # 替换成你的实际 API key
client = OpenAI(
    base_url="https://api2.aigcbest.top/v1",
    api_key=key
)

# 读取测试结果
json_file = "/content/drive/MyDrive/AIAA3102/Final_Project/Results/test_results.json"
with open(json_file, 'r', encoding='utf-8') as f:
    results = json.load(f)

def extract_json(text):
    try:
        # 匹配第一个大括号包裹的 JSON 对象
        match = re.search(r'\{.*\}', text, re.DOTALL)
        if match:
            return json.loads(match.group())
        else:
            return None
    except Exception as e:
        print("解析 JSON 失败:", e)
        return None

# 创建评分 prompt
def create_judge_prompt(prompt_text, generated_text):
    return f"""
你是心理咨询的评分专家，请根据以下维度对给定回答评分，每个维度 1-5 分（1 最差，5 最好），并严格输出 JSON 格式：

维度：
1. relevance: 回答是否直接回应了问题。
2. politeness: 回答是否礼貌、有同理心。
3. clarity: 回答是否易读、语言流畅。
4. usefulness: 提供的建议是否具体、可操作。
5. professionalism: 回答是否专业、安全、不会误导。

问题: {prompt_text}
模型回答: {generated_text}

请输出如下 JSON:
{{"relevance": , "politeness": , "clarity": , "usefulness": , "professionalism": }}
"""

# 批量评分
scores = []

for ex in tqdm(results, desc="LLM-as-Judge评分"):
    prompt_judge = create_judge_prompt(ex['prompt'], ex['generated'])
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt_judge}],
            temperature=0
        )
        content = response.choices[0].message.content
        score = extract_json(content)
        if score is None:
            print(f"❌ 样本 {ex['id']} JSON 解析失败，跳过")
            continue
        score['id'] = ex['id']
        scores.append(score)
    except Exception as e:
        print(f"❌ 样本 {ex['id']} 评分失败: {e}")
        continue

# 转成 DataFrame
df_scores = pd.DataFrame(scores)
print(df_scores.head())

# 计算平均分
mean_scores = df_scores[['relevance','politeness','clarity','usefulness','professionalism']].mean()
print("\n各维度平均分:")
print(mean_scores)

# 保存结果
output_file = "/content/drive/MyDrive/AIAA3102/Final_Project/Results/test_results_scores.json"
df_scores.to_json(output_file, orient='records', force_ascii=False, indent=2)
print(f"\n💾 评分结果已保存至: {output_file}")

LLM-as-Judge评分: 100%|██████████| 5/5 [00:07<00:00,  1.41s/it]

   relevance  politeness  clarity  usefulness  professionalism  id
0          2           4        3           2                3   0
1          3           4        3           3                3   1
2          4           5        4           4                5   2
3          2           3        3           2                2   3
4          5           5        5           4                5   4

各维度平均分:
relevance          3.2
politeness         4.2
clarity            3.6
usefulness         3.0
professionalism    3.6
dtype: float64

💾 评分结果已保存至: /content/drive/MyDrive/AIAA3102/Final_Project/Results/test_results_scores.json


In [ ]:
!pip install  rouge_score
import json
from rouge_score import rouge_scorer
import pandas as pd

# 读取测试结果
json_file = "/content/drive/MyDrive/AIAA3102/Final_Project/Results/test_results.json"
with open(json_file, 'r', encoding='utf-8') as f:
    results = json.load(f)

# 创建 ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

# 存放分数
rouge_scores = []

for ex in results:
    expected = ex.get('expected', '')
    generated = ex.get('generated', '')
    if expected.strip() == '' or generated.strip() == '':
        continue

    score = scorer.score(expected, generated)
    rougeL_f = score['rougeL'].fmeasure  # F1 分数
    rouge_scores.append({
        'id': ex['id'],
        'rougeL_f': rougeL_f
    })

# 转 DataFrame 查看
df_rouge = pd.DataFrame(rouge_scores)
print(df_rouge.head())

# 计算平均 ROUGE-L F1
mean_rougeL = df_rouge['rougeL_f'].mean()
print(f"\n平均 ROUGE-L F1 分数: {mean_rougeL:.4f}")

# 可选择保存结果
output_file = "/content/drive/MyDrive/AIAA3102/Final_Project/Results/test_results_rouge.json"
df_rouge.to_json(output_file, orient='records', force_ascii=False, indent=2)
print(f"💾 ROUGE-L 分数已保存至: {output_file}")

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=78e0c6303ef1fe1216d354b89d46eca3911255af42dc5030c308e8a88502661b
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score
   id  rougeL_f
0   0  0.153846
1   1  0.115502
2   2  0.140078
3   3  0.136170
4   4  0.135678

平均 ROUGE-L F1 分数: 0.1363
💾 ROUGE-L 分数已保存至: /content/drive/MyDrive/AIAA3102/Final_Project/Results/test_results_rouge.json
